In [20]:

from sage.crypto.boolean_function import BooleanFunction
import numpy as np
from multiprocess import Pool


In [21]:
n = 6

B = BooleanPolynomialRing(n, 'x', order = 'lex')
B.inject_variables()


Defining x0, x1, x2, x3, x4, x5


In [22]:
with open('/Users/lizaveta_yemets/Desktop/V3_combfunc.txt') as combining_function_file:
	combining_function_string = combining_function_file\
		.read()\
		.replace('_{', '')\
		.replace('} x', ' * x')\
		.replace('} +', ' +')

	combining_function = eval(combining_function_string)

combining_function


x0*x3*x5 + x1*x3*x5 + x1*x5 + x2*x3*x5 + x3*x4*x5 + x3*x5 + x3 + x4*x5

In [23]:
with open('/Users/lizaveta_yemets/Desktop/V3_gamma.txt') as validated_gamma_file:
	validated_gamma = validated_gamma_file.read()
	validated_gamma = [int(bit) for bit in validated_gamma]

validated_gamma[:10]


[1, 0, 1, 1, 0, 1, 0, 0, 1, 1]

In [24]:
def poly_to_int(poly):
    # Получаем список коэффициентов многочлена, исключая константный член
    coefficients = poly.list()[:-1]

    # Преобразуем каждый коэффициент в строку и объединяем их
    binary_string = ''.join([str(coef) for coef in coefficients])

    # Преобразуем двоичную строку в целое число с базой 2
    integer_value = int(binary_string, 2)

    return integer_value

polynomials = [
	x^10 + x^3 + 1,
	x^10 + x^7 + 1,
	x^10 + x^4 + x^3 + x + 1,
	x^10 + x^8 + x^3 + x^2 + 1,
	x^10 + x^8 + x^4 + x^3 + 1,
	x^10 + x^8 + x^5 + x + 1,
]
# Создание полиномов для аффинных функций
polynomials_int = [poly_to_int(poly) for poly in polynomials]

polynomials_int

[576, 516, 864, 706, 610, 786]

In [25]:
walsh_spectrum = list(BooleanFunction(combining_function).walsh_hadamard_transform())

print(walsh_spectrum)

[0, 0, 0, 0, 0, 16, 0, 0, 32, 0, 0, 0, 0, -16, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -16, 0, 0, 32, 0, 0, 0, 0, 16, 0, 0, 0, 0, -16, 0, 0, 0, 0, 0, 0, 0, -16, 0, 0, 0, 0, 0]


In [26]:
# Допоміжна функція для перетворення індексу в двійкове представлення
def int_to_binary(n, length=6):
    return [int(x) for x in bin(n)[2:].zfill(length)]

# Допоміжна функція для визначення знака числа
def sign(num):
    return -1 if num < 0 else 1

# Змінні для зберігання спектру Уолша
stringified_expressions = []
expressions = {}
expressions_with_walsh = {}

# Генерація аналогів на основі спектру Уолша
for index in range(len(walsh_spectrum)):
    coefficient = walsh_spectrum[index]
    analog_terms = []

    # Отримуємо двійкове представлення індексу
    index_binary = int_to_binary(index)

    # Додаємо змінні відповідно до двійкового індексу
    for i, binary_digit in enumerate(index_binary):
        if binary_digit == 1:
            analog_terms.append(f'x{i}')

    # Додаємо константу '1' при негативному коефіцієнті
    if sign(coefficient) == -1:
        analog_terms.append('1')

    # Об'єднуємо терміни в рядок і додаємо до списку аналогів
    if len(analog_terms) > 0:
        stringified_expression = ' + '.join(analog_terms)

        stringified_expressions.append(stringified_expression)
        expressions[stringified_expression] = BooleanFunction(eval(stringified_expression))
        expressions_with_walsh[stringified_expression] = coefficient

# Результат: список виразів
stringified_expressions


['x5',
 'x4',
 'x4 + x5',
 'x3',
 'x3 + x5',
 'x3 + x4',
 'x3 + x4 + x5',
 'x2',
 'x2 + x5',
 'x2 + x4',
 'x2 + x4 + x5',
 'x2 + x3',
 'x2 + x3 + x5 + 1',
 'x2 + x3 + x4',
 'x2 + x3 + x4 + x5',
 'x1',
 'x1 + x5',
 'x1 + x4',
 'x1 + x4 + x5',
 'x1 + x3',
 'x1 + x3 + x5',
 'x1 + x3 + x4',
 'x1 + x3 + x4 + x5',
 'x1 + x2',
 'x1 + x2 + x5',
 'x1 + x2 + x4',
 'x1 + x2 + x4 + x5',
 'x1 + x2 + x3',
 'x1 + x2 + x3 + x5',
 'x1 + x2 + x3 + x4',
 'x1 + x2 + x3 + x4 + x5',
 'x0',
 'x0 + x5',
 'x0 + x4',
 'x0 + x4 + x5',
 'x0 + x3',
 'x0 + x3 + x5 + 1',
 'x0 + x3 + x4',
 'x0 + x3 + x4 + x5',
 'x0 + x2',
 'x0 + x2 + x5',
 'x0 + x2 + x4',
 'x0 + x2 + x4 + x5',
 'x0 + x2 + x3',
 'x0 + x2 + x3 + x5',
 'x0 + x2 + x3 + x4',
 'x0 + x2 + x3 + x4 + x5',
 'x0 + x1',
 'x0 + x1 + x5',
 'x0 + x1 + x4 + 1',
 'x0 + x1 + x4 + x5',
 'x0 + x1 + x3',
 'x0 + x1 + x3 + x5',
 'x0 + x1 + x3 + x4',
 'x0 + x1 + x3 + x4 + x5',
 'x0 + x1 + x2',
 'x0 + x1 + x2 + x5',
 'x0 + x1 + x2 + x4 + 1',
 'x0 + x1 + x2 + x4 + x5',
 'x0 +

In [27]:
boolean_combining_function = BooleanFunction(combining_function)

# Ініціалізація ймовірностей для кожного аналогового виразу
probabilities = dict(zip(stringified_expressions, np.zeros(len(stringified_expressions))))

num_states = 1 << n


# Обчислення ймовірності для кожного аналогового виразу
for stringified_expression in stringified_expressions:
    analog_function = expressions[stringified_expression]
    for state in range(num_states):
        if boolean_combining_function(state) == analog_function(state):
            probabilities[stringified_expression] += 1

# Перетворення значень у ймовірності
for expression in probabilities.keys():
    probabilities[expression] /= num_states

# Сортування за ймовірностями
sorted_probabilities = dict(sorted(probabilities.items(), key=lambda x: x[1], reverse=True))

# Виведення результату
print("Ймовірності аналогів:")
for expression, probability in sorted_probabilities.items():
    print(f"{expression}: {probability:.2f}")



Ймовірності аналогів:
x3: 0.75
x3 + x5: 0.75
x1 + x4: 0.62
x1 + x3 + x4: 0.62
x0 + x2: 0.62
x0 + x2 + x3 + x5: 0.62
x5: 0.50
x4: 0.50
x4 + x5: 0.50
x3 + x4: 0.50
x3 + x4 + x5: 0.50
x2: 0.50
x2 + x5: 0.50
x2 + x4: 0.50
x2 + x4 + x5: 0.50
x2 + x3: 0.50
x2 + x3 + x5 + 1: 0.50
x2 + x3 + x4: 0.50
x2 + x3 + x4 + x5: 0.50
x1: 0.50
x1 + x5: 0.50
x1 + x3: 0.50
x1 + x3 + x5: 0.50
x1 + x2: 0.50
x1 + x2 + x5: 0.50
x1 + x2 + x4: 0.50
x1 + x2 + x4 + x5: 0.50
x1 + x2 + x3: 0.50
x1 + x2 + x3 + x5: 0.50
x1 + x2 + x3 + x4: 0.50
x1 + x2 + x3 + x4 + x5: 0.50
x0: 0.50
x0 + x5: 0.50
x0 + x4: 0.50
x0 + x4 + x5: 0.50
x0 + x3: 0.50
x0 + x3 + x5 + 1: 0.50
x0 + x3 + x4: 0.50
x0 + x3 + x4 + x5: 0.50
x0 + x2 + x4: 0.50
x0 + x2 + x4 + x5: 0.50
x0 + x2 + x3 + x4: 0.50
x0 + x2 + x3 + x4 + x5: 0.50
x0 + x1: 0.50
x0 + x1 + x5: 0.50
x0 + x1 + x4 + 1: 0.50
x0 + x1 + x4 + x5: 0.50
x0 + x1 + x3: 0.50
x0 + x1 + x3 + x5: 0.50
x0 + x1 + x3 + x4: 0.50
x0 + x1 + x3 + x4 + x5: 0.50
x0 + x1 + x2: 0.50
x0 + x1 + x2 + x5: 0.50
x0 +

In [28]:
# Отримання найкращих виразів за ймовірністю
top_expressions = list(sorted_probabilities.keys())[:10]


# Збір булевих функцій для найкращих виразів
top_candidates = [expressions[stringified_expressions] for stringified_expressions in top_expressions]

# Виведення списку найкращих виразів
top_expressions

['x3',
 'x3 + x5',
 'x1 + x4',
 'x1 + x3 + x4',
 'x0 + x2',
 'x0 + x2 + x3 + x5',
 'x5',
 'x4',
 'x4 + x5',
 'x3 + x4']

In [29]:
# Calculate weights and coefficients for correlation
weights = [sum(list(map(int, bin(i)[2:]))) for i in range(2**6)]
sorted_weights, sorted_coefs = zip(*sorted(zip(weights, walsh_spectrum)))

# Find and print the correlation
for i in range(2**6):
    if sorted_coefs[i] != 0:
        print('cor(f) =', sorted_weights[i] - 1)
        break

cor(f) = 0


In [31]:
%%cython

from libc.stdlib cimport malloc, free
cdef int LENGTH = 10

def generate_sequence(int poly, int init_state, int nbits):
	cdef int curr_state = init_state, s, i, j
	cdef int* res_array = <int*>malloc(sizeof(int)*nbits)

	try:
		for i in range(nbits):
			res_array[i] = (curr_state >> (LENGTH - 1)) & 1
			s = 0
			
			for j in range(LENGTH):
				s = s ^ (((curr_state & poly) >> j) & 1)
			
			curr_state = (curr_state << 1) | s
			
		return [x for x in res_array [:nbits]]
	finally:
		free(res_array)


ld: warning: passed two min versions (11.0, 11.0) for platform macOS. Using 11.0.
ld: warning: passed two min versions (11.0, 11.0) for platform macOS. Using 11.0.


In [32]:
def state_to_int(*bits):
	number = 0

	for index, bit in bits:
		temp = bit << index

		number |= temp

	return number


In [33]:

# Потужність для повного перебору
bruteforce_power = 1 << 10

# Отримання вектору бітів із заданих пар (індекс, біт)
def get_bit_vector(*params, length=6):
    vector = [0] * length
    for index, bit in params:
        vector[index] = bit
    return vector

# Обчислення значення T для заданого коефіцієнта k
def get_T(k):
    return int(8 * 0.5**-2 * np.log(2**(10 * k) * 100))

# Отримання кількості відмінностей між двома бітовими послідовностями
def get_bit_difference(first_sequence, second_sequence):
    return sum(a != b for a, b in zip(first_sequence, second_sequence))

# Сортування словника за значеннями
def get_sorted_dict(dictionary):
    return sorted(dictionary.items(), key=lambda x: x[1])

In [34]:


# Обчислення різниці для одного набору початкових даних
def bruteforce_single_variable(init_state, polynomial_index, g_function, sequence_length):
    polynomial = polynomials_int[polynomial_index]
    generated_sequence = generate_sequence(polynomial, init_state, sequence_length)
    test_sequence = [g_function(state_to_int((polynomial_index, bit))) for bit in generated_sequence]

    return init_state, get_bit_difference(test_sequence, validated_gamma[:sequence_length])




In [59]:
%%time

g_0 = top_candidates[0]
T_0 = get_T(1)

print(f'g_0 = {top_expressions[0]}')
print(f'')
print(f'T = {T_0}')
print(f'----------')


with Pool() as pool:
	differences_0 = dict(
		pool.map(
			lambda init_state: bruteforce_single_variable(init_state, 3, g_0, T_0),
			range(bruteforce_power)
		)
	)


sorted_differences = get_sorted_dict(differences_0)[:10]
for initial_state, hemming_distance in sorted_differences:
    print(f"Initial State: {initial_state}, Hemming Distance: {hemming_distance}")


g_0 = x3

T = 369
----------
Initial State: 595, Hemming Distance: 91
Initial State: 195, Hemming Distance: 157
Initial State: 717, Hemming Distance: 157
Initial State: 710, Hemming Distance: 158
Initial State: 37, Hemming Distance: 160
Initial State: 947, Hemming Distance: 160
Initial State: 977, Hemming Distance: 160
Initial State: 703, Hemming Distance: 161
Initial State: 454, Hemming Distance: 162
Initial State: 742, Hemming Distance: 162
CPU times: user 14 ms, sys: 51.3 ms, total: 65.3 ms
Wall time: 143 ms


In [41]:
value_3 = 595

In [42]:

# Функція брутфорсу для однієї змінної з врахуванням заданої послідовності
def bruteforce_one_variable_custom(init_state, second_polynomial_index, g_function, sequence_length, first_polynomial_data):
    first_polynomial_index, first_sequence = first_polynomial_data
    second_polynomial = polynomials_int[second_polynomial_index]
    second_sequence = generate_sequence(second_polynomial, init_state, sequence_length)

    # Створення тестової послідовності
    test_sequence = [
        g_function(state_to_int((first_polynomial_index, first_sequence[i]), (second_polynomial_index, second_sequence[i])))
        for i in range(sequence_length)
    ]

    # Повертає початковий стан та відмінність Геммінга між тестовою та валідаційною послідовностями
    return init_state, get_bit_difference(test_sequence, validated_gamma[:sequence_length])



In [44]:
%%time

g_1 = top_candidates[1]
T_1 = get_T(2)

print(f'g_1 = {top_expressions[1]}')
print(f'')
print(f'T = {T_1}')
print(f'----------')

sequence_poly_3 = generate_sequence(polynomials_int[3], value_3, T_1)

with Pool() as pool:
	differences_1 = dict(
		pool.map(
			lambda init_state: bruteforce_one_variable_custom(init_state, 5, g_1, T_1, (3, sequence_poly_3)),
			range(bruteforce_power)
		)
	)


sorted_differences = get_sorted_dict(differences_1)[:10]
for initial_state, hemming_distance in sorted_differences:
    print(f"Initial State: {initial_state}, Hemming Distance: {hemming_distance}")


g_1 = x3 + x5

T = 590
----------
Initial State: 0, Hemming Distance: 148
Initial State: 680, Hemming Distance: 149
Initial State: 82, Hemming Distance: 269
Initial State: 145, Hemming Distance: 269
Initial State: 528, Hemming Distance: 269
Initial State: 603, Hemming Distance: 269
Initial State: 949, Hemming Distance: 269
Initial State: 727, Hemming Distance: 270
Initial State: 902, Hemming Distance: 270
Initial State: 909, Hemming Distance: 270
CPU times: user 18.9 ms, sys: 43.1 ms, total: 62 ms
Wall time: 200 ms


In [45]:
value_5 = 680

In [46]:
# Обчислення різниці для двох змінних з врахуванням заданих поліномів
def brutforce_for_two_variable(init_state_1, polynomials_indexes, g, T):
	poly_index_1, poly_index_2 = polynomials_indexes

	differences = []
     # Генерація першої послідовності
	sequence_1 = generate_sequence(polynomials_int[poly_index_1], init_state_1, T)
    # Перебір другого початкового стану
	for init_state_2 in range(bruteforce_power):
		sequence_2 = generate_sequence(polynomials_int[poly_index_2], init_state_2, T)
         # Створення тестової послідовності
		test_sequence = [
			g(state_to_int((poly_index_1, sequence_1[i]), (poly_index_2, sequence_2[i]))) 
			for i in range(T)
		]

		differences.append(((init_state_1, init_state_2), get_bit_difference(test_sequence, validated_gamma[:T])))

	return differences



In [51]:
%%time

g_2 = top_candidates[4]
T_2 = get_T(3)

print(f'g_2 = {top_expressions[4]}')
print(f'')
print(f'T = {T_2}')
print(f'----------')

with Pool() as pool:
	differences_2 = list(
		pool.map(
			lambda init_state_1: brutforce_for_two_variable(init_state_1, (0, 2), g_2, T_2),
			range(bruteforce_power)
		)
	)

	differences_2 = dict([difference for sublist in differences_2 for difference in sublist])

sorted_differences = get_sorted_dict(differences_2)[:10]
for initial_state, hemming_distance in sorted_differences:
    print(f"Initial State: {initial_state}, Hemming Distance: {hemming_distance}")



g_2 = x0 + x2

T = 812
----------
Initial State: (498, 1004), Hemming Distance: 292
Initial State: (883, 642), Hemming Distance: 334
Initial State: (888, 178), Hemming Distance: 336
Initial State: (538, 159), Hemming Distance: 339
Initial State: (535, 684), Hemming Distance: 341
Initial State: (808, 299), Hemming Distance: 341
Initial State: (989, 413), Hemming Distance: 342
Initial State: (623, 878), Hemming Distance: 343
Initial State: (836, 943), Hemming Distance: 343
Initial State: (892, 437), Hemming Distance: 343
CPU times: user 884 ms, sys: 284 ms, total: 1.17 s
Wall time: 2min 44s


In [52]:
value_0 = 498
value_2 = 1004

In [54]:
%%time

g_3 = top_candidates[2]
T_3 = get_T(3)

print(f'g_3 = {top_expressions[2]}')
print(f'')
print(f'T = {T_3}')
print(f'----------')

with Pool() as pool:
	differences_3 = list(
		pool.map(
			lambda init_state_1: brutforce_for_two_variable(init_state_1, (1, 4), g_3, T_3),
			range(bruteforce_power)
		)
	)

	differences_3 = dict([difference for sublist in differences_3 for difference in sublist])

sorted_differences = get_sorted_dict(differences_3)[:10]
for initial_state, hemming_distance in sorted_differences:
    print(f"Initial State: {initial_state}, Hemming Distance: {hemming_distance}")


g_3 = x1 + x4

T = 812
----------
Initial State: (882, 224), Hemming Distance: 295
Initial State: (71, 208), Hemming Distance: 341
Initial State: (112, 811), Hemming Distance: 341
Initial State: (0, 638), Hemming Distance: 342
Initial State: (953, 377), Hemming Distance: 342
Initial State: (200, 383), Hemming Distance: 343
Initial State: (363, 752), Hemming Distance: 343
Initial State: (417, 792), Hemming Distance: 343
Initial State: (960, 561), Hemming Distance: 343
Initial State: (143, 338), Hemming Distance: 345
CPU times: user 957 ms, sys: 385 ms, total: 1.34 s
Wall time: 3min 16s


In [55]:
value_1 = 882
value_4 = 224

In [57]:
l = 50000

# Generate sequences
sequences = [generate_sequence(polynomials_int[i], value, l) for i, value in enumerate([value_0, value_1, value_2, value_3, value_4, value_5])]

# Generate the combined sequence using boolean_combining_function
sequence = [
    int(boolean_combining_function(state_to_int(
        (0, sequences[0][i]), 
        (1, sequences[1][i]), 
        (2, sequences[2][i]), 
        (3, sequences[3][i]), 
        (4, sequences[4][i]), 
        (5, sequences[5][i])
    )))
    for i in range(l)
]

# Print the first 15 elements of the sequence and validated_gamma
print(sequence[:15])
print(validated_gamma[:15])

# Calculate and print the bit difference
bit_difference = get_bit_difference(sequence, validated_gamma[:l])
print(bit_difference)


[1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1]
[1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1]
0
